In [17]:
import re, time, json
from dataclasses import dataclass
from pathlib import Path
from typing import List, Tuple

In [18]:
DOC_PATH = Path("../doc/RDPro.md")
PY_PATH  = Path("../python/ndvi.py")
assert DOC_PATH.exists(), f"Missing: {DOC_PATH}"
assert PY_PATH.exists(), f"Missing: {PY_PATH}"

rdpro_text = DOC_PATH.read_text(encoding="utf-8")
py_code = PY_PATH.read_text(encoding="utf-8")

In [19]:
MODEL = "gpt-5"

SYSTEM_PROMPT = """
You are a geospatial data engineer and Spark systems expert.

Task: Convert a given geospatial Python script into Scala code that runs on RDPro (Spark-based raster processing) on Apache Spark.

You must understand Spark execution and produce distributed, RDD-based Scala.

AUTHORITATIVE EXAMPLES RULE:
- For each operation, Scala/Python examples may exist under examples/.
- If an example file is missing or empty, ignore it.
- Use only APIs/signatures shown in DOC CHUNKS OR non-empty examples.
- Never assume Scala and Python examples are symmetric.

Environment & paths:
- Determine whether output paths should be treated as local or distributed based on Spark configuration and the URI scheme.
- You MAY use standard Spark/Scala APIs for this (SparkConf, SparkContext.hadoopConfiguration, java.net.URI, java.nio.file).
- You MUST NOT invent any RDPro path utilities.

FILESYSTEM & PATH NORMALIZATION (MANDATORY):
- Detect Spark local mode using SparkContext:
  - Treat as local if `sc.master` starts with "local" (case-insensitive).
- Before calling any RDPro IO API (e.g., geoTiff read/write), normalize ALL input/output paths:
  1) If the path already has a URI scheme (file:, hdfs:, s3a:, gs:, http:, etc.), use it as-is.
  2) If the path has NO scheme AND Spark is local AND the path looks like a local filesystem path
     (e.g., starts with "/" on Unix/macOS, or has a Windows drive like "C:\\"), convert it to an
     absolute `file:///...` URI using standard Java APIs (java.net.URI + java.nio.file.Paths).
  3) If Spark is NOT local, do NOT prepend file:///; leave scheme-less paths unchanged so they
     resolve against the cluster filesystem config (fs.defaultFS).
- This rule exists to prevent Hadoop from interpreting local absolute paths as HDFS
  (e.g., hdfs://localhost:9000).
  
Hard rules:
1) Output MUST be valid Scala and compile as an RDPro operation module.
   Required structure:
   - `object <OperationName> { def run(sc: SparkContext): <ReturnType> = ... }`
   - Include all necessary imports
   - Do NOT define `main` and do NOT use `extends App`
   - Do NOT create or stop SparkSession or SparkContext inside `run`
   - Assume SparkContext `sc` is provided by the caller
2) Use ONLY RDPro APIs that appear in the provided DOC.
   - If a method signature is not shown in DOC, do NOT guess.
3) Do NOT invent RDPro APIs, overloads, implicits, or helper utilities. No hidden "magic" conversions.
4) Preserve semantics of the Python: raster IO, pixel math, focal ops, masking/nodata, reprojection/resample if present.
5) Distributed correctness:
   - Avoid driver-side operations: do NOT call collect/toLocalIterator unless required by the Python semantics.
   - Prefer RDPro RasterRDD end-to-end when available in DOC .
6) Raster alignment robustness:
   - If not in DOC, fail fast: throw a clear runtime error explaining alignment is required but unsupported with available APIs.
7) Performance guidance (Spark-level only):
   - You MAY set Spark SQL / Spark configs and use standard Spark operations (repartition/coalesce/cache/persist) ONLY when:
     (a) it does not change semantics, and
     (b) it is justified by an obvious pipeline boundary (e.g., before a wide op / expensive reuse).
8) Lambdas:
   - When passing lambdas to RDPro functions (e.g., mapPixels), add explicit parameter and return types so Scala compiles.
9) CLI args:
   - If the Python has input/output paths, read them from args with safe defaults and validation.
   - Do not introduce extra parameters not implied by the Python.

Output format (strict):
- First: Scala file content only (NO markdown fences).
- After the Scala: a "NOTES" section listing:
  (a) RDPro APIs used (names only)
  (b) Unsupported operations and why (especially if missing alignment/warp APIs)
  (c) Assumptions about IO paths / bands / nodata / CRS / environment detection logic
""".strip()

In [20]:
def build_user_prompt(doc_pack: str, py_code: str) -> str:
    return f"""
RDPro documentation (relevant DOC CHUNKS only):
{doc_pack}

Python script:
{py_code}

Task:
Translate the Python script into Scala targeting RDPro on Spark.
Use ONLY APIs described in the DOC CHUNKS.
""".strip()

In [21]:
# Cell 7 — LLM call stub (you plug in your model call here)
def call_llm(prompt: str) -> str:
    """
    Replace this with your real LLM call.
    It should return Scala code as a string.
    """
    raise NotImplementedError("Plug in your LLM API call here.")

# Example usage:
# scala_code_manual = call_llm(prompt_manual)
# scala_code_auto   = call_llm(prompt_auto)


In [22]:
OUT_DIR = Path("./runs/workspace")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [23]:
user_prompt = build_user_prompt(rdpro_text, py_code)

In [24]:
import os

if "OPENAI_API_KEY" in os.environ:
    print("OPENAI_API_KEY is set")
else:
    print("OPENAI_API_KEY is NOT set")

OPENAI_API_KEY is set


In [26]:
from openai import OpenAI
client = OpenAI()

def run_llm(prompt: str) -> Tuple[str, float]:
    t0 = time.time()
    resp = client.responses.create(
        model=MODEL,
        input=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
    )
    return resp.output_text.strip(), time.time() - t0

scala_out, dt = run_llm(user_prompt)
(OUT_DIR / "ndvi_doc_all_GPT_2.scala").write_text(scala_out, encoding="utf-8")

print("Wrote:", OUT_DIR / "ndvi_doc_all_GPT_2.scala")
print("LLM latency:", round(dt, 2), "s")
print("\n--- Preview ---\n", scala_out[:900])

Wrote: runs/workspace/ndvi_doc_all_GPT_2.scala
LLM latency: 118.86 s

--- Preview ---
 import org.apache.spark.SparkContext
import edu.ucr.cs.bdlab.beast._
import edu.ucr.cs.bdlab.raptor.GeoTiffWriter
import edu.ucr.cs.bdlab.beast.io.tiff.TiffConstants
import java.net.URI
import java.nio.file.Paths

object ComputeNDVI {
  private def hasScheme(path: String): Boolean = {
    try {
      val s = new URI(path).getScheme
      s != null && s.nonEmpty
    } catch {
      case _: Throwable => false
    }
  }

  private def isLocalMaster(sc: SparkContext): Boolean = {
    val m = Option(sc.master).getOrElse("")
    m.toLowerCase.startsWith("local")
  }

  private def looksLikeLocalAbsolute(path: String): Boolean = {
    try {
      Paths.get(path).isAbsolute
    } catch {
      case _: Throwable => false
    }
  }

  private def normalizePath(sc: SparkContext, path: String): String = {
    if (hasScheme(path)) {
      path
    } else if (isLocalMaster(sc) && looksLikeLocalAbsolut
